In [ ]:
!pip install tensorflow==2.11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 22 kB/s 
     |████████████████████████████████| 6.0 MB 62.9 MB/s 
     |████████████████████████████████| 1.7 MB 75.2 MB/s 
     |████████████████████████████████| 439 kB 73.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.10.0
    Uninstalling tensorflow-2.10.0:
      Succ

In [ ]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.layers import Layer
from tensorflow.python.framework.ops import disable_eager_execution

2.10.0


In [ ]:
non_eager_execution = False
if non_eager_execution:
   tf.compat.v1.disable_eager_execution()
else:
   tf.config.run_functions_eagerly(True)

print("Eager execution is: " + str(tf.executing_eagerly()))

Eager execution is: True


In [ ]:
class RNN_simple_layer(Layer):

 def __init__(self, neurons_numbers, return_sequences=False,**kwargs):
    super(RNN_simple_layer, self).__init__()
    self.neurons_numbers  = neurons_numbers
    self.return_sequences = return_sequences

 def build(self, input_shape):
    self.kernel_input = self.add_weight('kernel_input', shape=[int(input_shape[-1]), self.neurons_numbers], initializer="random_normal",trainable=True)
    self.kernel_state = self.add_weight('kernel_state', shape=[self.neurons_numbers,self.neurons_numbers], initializer="random_normal",trainable=True)
    self.bias = self.add_weight('bias', shape=[self.neurons_numbers,],initializer="zeros",trainable=True)

 def call(self, inputs):

    sequence_length = inputs.shape[-2]

    # outputs = []
    outputs = tf.TensorArray(tf.float32, size=sequence_length)
    states  = tf.TensorArray(tf.float32, size=sequence_length)
    inital_states  = tf.zeros(shape=[tf.shape(inputs)[0],self.neurons_numbers])

    states = inital_states
    for i_step in range(sequence_length):
        output_i_step = tf.tanh(tf.matmul(inputs[:,i_step,:],self.kernel_input) + tf.matmul(states, self.kernel_state) + self.bias)
        states        = output_i_step
        outputs = outputs.write(i_step,output_i_step)

    # output the RNN resuts
    if self.return_sequences == True:
        output = tf.transpose(outputs.stack(), [1,0,2])
    else:
        output = outputs.stack()[-1]
    return output
#----------------------------------------------

#(2) Test the customized simple RNN layer
#----------------------------------------------
RNN_layer = RNN_simple_layer(4,return_sequences=True)

In [ ]:
input_array = tf.ones((32,10,8))
output_array = RNN_layer(input_array)

print('RNN input tensor shape:' + str(input_array.shape))
print('RNN input tensor has batch size:' + str(input_array.shape[0]))
print('RNN input tensor has time samples:' + str(input_array.shape[-2]))
print('RNN input tensor has features of each sample:' + str(input_array.shape[-1]))
print('RNN output tensor shape:' + str(output_array.shape))

RNN input tensor shape:(32, 10, 8)
RNN input tensor has batch size:32
RNN input tensor has time samples:10
RNN input tensor has features of each sample:8
RNN output tensor shape:(32, 10, 4)


In [ ]:
inputs = tf.keras.Input(shape=(10,20))
x = tf.keras.layers.Conv1D(10,3)(inputs)
x = RNN_simple_layer(4,return_sequences=True)(x)
outputs = tf.keras.layers.Dense(30, activation="relu")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="test_model")
model.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 20)]          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 8, 10)             610       
                                                                 
 rnn_simple_layer_3 (RNN_sim  (None, 8, 4)             60        
 ple_layer)                                                      
                                                                 
 dense_1 (Dense)             (None, 8, 30)             150       
                                                                 
Total params: 820
Trainable params: 820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save('/content/model')

In [ ]:

#if we somehow set a static input shape for the model, it would work.
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('/content/model')
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float32]
converter.optimize = [tf.lite.Optimize.DEFAULT]
#converter.experimental_new_converter = True
#converter.experimental_new_quantizer = False
#converter.experimental_enable_resource_variables = True
tflite_model_file = 'Model_simple_RNN_TF28.tflite'

tflite_model = converter.convert()

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

ConverterError: ignored